In [1]:
import pandas as pd
import numpy as np
import hashlib
import base64
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Preprocessing notebook for PheKnowLator's Human Phenotype Ontology (HPO) - Mouse Phenotype (MP) Ontology mappings

In [2]:
#!jupyter nbconvert --to script tiffanys_mappings_JS.ipynb

In [26]:
def fill_missing_cols(df):
    
    if 'node_id' not in df.columns: 
        raise ValueError('Must have at least a "node_id" column.')
        
    all_cols = set([ 'node_label', 'node_synonyms', 'node_dbxrefs',
            'node_definition','node_namespace','value','lowerbound','upperbound','unit'])
    
    missing_cols = list(all_cols - set(df.columns))
    nan_cols_df = pd.DataFrame(np.full([len(df), len(missing_cols)], np.nan),columns=missing_cols)

    if isinstance(df, pd.DataFrame):
        nan_cols_df.index = df.index
        return pd.concat([df,nan_cols_df],axis=1)
    elif isinstance(df, pl.DataFrame):
        # no index for polars
        return pl.concat([df,pl.from_pandas(nan_cols_df)],how='horizontal')
    else:
        raise ValueError(f'Must Pass either a pandas DataFrame or a polars DataFrame but recieved "{type(df)}".')


# This Notebook creates relationships between HPO and MP Concept nodes

# The end of this workflow is different from the original tiffanys_mappings.ipynb notebook located in /Users/stearb/Dropbox/CHOP/R03/code/phenotype_mapping, we are using Jonathan Silversteins workflow for the Neo4j CSV creation (meaning the files produced by this workflow will be the inputs into JS's workflow) ...so we only need to create 2 files, a nodes.tsv and an edges.tsv (instead of the ~6 files, CUIs, CUI-CUIs, Code-CUIs, Terms, etc.)

## The guide for how to create these new nodes and edges files can be found in the Data Distillerys [github](https://ubkg.docs.xconsortia.org/formats/)

In [3]:
df=pd.read_excel('/Users/stearb/desktop/DESKTOP_TRANSFER/R03_local/data/tiffany_mappings/KF_HPO_OBO2OBOMappings_Aggregated_20DEC2020.xlsx',                                                            sheet_name='KF_OMOP2OBMappings_20Dec2020')
df.head(4)

,HP_ID,HP_LABEL,HP_SYNONYM,MPO_URI,MPO_LABEL,MPO_MAPPING,MPO_EVIDENCE
0,HP_0000011,neurogenic bladder,lack of bladder control due to nervous system ...,MP_0005302,neurogenic bladder,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_LABEL:neurogenic bladder...
1,HP_0000023,inguinal hernia,NaN,MP_0006077,inguinal hernia,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_LABEL:inguinal hernia | ...
2,HP_0000028,cryptorchidism,undescended testes | undescended testis | cryp...,MP_0002286,cryptorchism,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_SYNONYM_hasExactSynonym:...
3,HP_0000033,"ambiguous genitalia, male",ambiguous genitalia in males,MP_0002160 | MP_0001145,abnormal reproductive system morphology | abno...,Automatic Constructor Ancestor,MPO_LABEL-HPO_ANCESTOR_LABEL:abnormal reproduc...


In [4]:
# Replace _ with :
df['HP_ID']  = df['HP_ID'].str.replace('_', ':')
df['MPO_URI']  = df['MPO_URI'].str.replace('_', ':')


# Splitby '|' so we can  have one HPO term and one MP term per row so we can upload to neo4j database.
df['MPO_URI'] = df['MPO_URI'].str.split('|')
df['MPO_URI'].head(5)

0                  [MP:0005302]
1                  [MP:0006077]
2                  [MP:0002286]
3    [MP:0002160 ,  MP:0001145]
4                  [MP:0003623]
Name: MPO_URI, dtype: object

In [6]:
df.head(4) # still have multiple MP

,HP_ID,HP_LABEL,HP_SYNONYM,MPO_URI,MPO_LABEL,MPO_MAPPING,MPO_EVIDENCE
0,HP:0000011,neurogenic bladder,lack of bladder control due to nervous system ...,[MP:0005302],neurogenic bladder,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_LABEL:neurogenic bladder...
1,HP:0000023,inguinal hernia,NaN,[MP:0006077],inguinal hernia,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_LABEL:inguinal hernia | ...
2,HP:0000028,cryptorchidism,undescended testes | undescended testis | cryp...,[MP:0002286],cryptorchism,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_SYNONYM_hasExactSynonym:...
3,HP:0000033,"ambiguous genitalia, male",ambiguous genitalia in males,"[MP:0002160 , MP:0001145]",abnormal reproductive system morphology | abno...,Automatic Constructor Ancestor,MPO_LABEL-HPO_ANCESTOR_LABEL:abnormal reproduc...


In [7]:
# Unravel MPO_URI column
df_ravel = pd.DataFrame(columns=df.columns)
j=0

for index, row in df.iterrows():    
    if len(row['MPO_URI']) == 1:
        df_ravel.loc[j] = row
        j=j+1
       
    elif len(row['MPO_URI']) > 1:
        current_row = row.drop(['MPO_URI']).T # Get all values except MPO_URI (which contains multiple terms)
        
        for MPO_TERM in row['MPO_URI']:
            row_temp = current_row
            mpo_formatted = pd.DataFrame([MPO_TERM],columns=['MPO_URI']).T
            mpo_formatted = mpo_formatted[0]
            new_row = row_temp._append(mpo_formatted).T # combine row_ with each MPO_URI and add them as new rows
            new_row=new_row[list(df_ravel.columns)]

            df_ravel.loc[j]  = np.ravel(new_row.T.values)
            j=j+1
            row_temp = 0


In [8]:
df_ravel.head(3) # Need convert the MPO_URIs that are lists of length 1, to single string values

,HP_ID,HP_LABEL,HP_SYNONYM,MPO_URI,MPO_LABEL,MPO_MAPPING,MPO_EVIDENCE
0,HP:0000011,neurogenic bladder,lack of bladder control due to nervous system ...,[MP:0005302],neurogenic bladder,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_LABEL:neurogenic bladder...
1,HP:0000023,inguinal hernia,NaN,[MP:0006077],inguinal hernia,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_LABEL:inguinal hernia | ...
2,HP:0000028,cryptorchidism,undescended testes | undescended testis | cryp...,[MP:0002286],cryptorchism,Automatic Exact Concept,MPO_LABEL-HPO_CONCEPT_SYNONYM_hasExactSynonym:...


In [9]:
# Some of the values in the MPO_URI column are strings and some are lists (of one string), 
# we need to make them all strings...
Counter([type(i) for i in df_ravel['MPO_URI']])

Counter({list: 443, str: 778})

In [10]:

mp_fixed = []
for i in df_ravel['MPO_URI']:
    if len(i) == 1: # Length is 1 if its a list, otherwise it's the length of the string
        mp_fixed.append(i[0])
    else:
        mp_fixed.append(i)

assert df_ravel.shape[0] == len(mp_fixed)
df_ravel['MPO_URI'] = mp_fixed

In [11]:
# Now its fixed (all strings, no lists)
Counter([type(i) for i in df_ravel['MPO_URI']])

Counter({str: 1221})

In [12]:
print('Number of unique HPO terms: '+str(df_ravel['HP_ID'].unique().shape[0]))
print('Number of unique MP terms: '+str(df_ravel['MPO_URI'].unique().shape[0]))

Number of unique HPO terms: 719
Number of unique MP terms: 602


In [13]:
df_ravel.drop(['MPO_MAPPING','MPO_EVIDENCE'],axis=1,inplace=True) # Remove these cols for now

In [14]:
# Strip white space from mp terms
df_ravel['MPO_URI']  = [i.strip() for i in df_ravel['MPO_URI']]

In [15]:
df_ravel.head(4)

,HP_ID,HP_LABEL,HP_SYNONYM,MPO_URI,MPO_LABEL
0,HP:0000011,neurogenic bladder,lack of bladder control due to nervous system ...,MP:0005302,neurogenic bladder
1,HP:0000023,inguinal hernia,NaN,MP:0006077,inguinal hernia
2,HP:0000028,cryptorchidism,undescended testes | undescended testis | cryp...,MP:0002286,cryptorchism
3,HP:0000033,"ambiguous genitalia, male",ambiguous genitalia in males,MP:0002160,abnormal reproductive system morphology | abno...


In [16]:
#df_ravel.to_csv('/Users/stearb/Desktop/R03_local/data/tiffany_mappings/pheno_mappings.csv',index=False)

## Need to Connect these at the Concept & Code level, so we need to add CUIs, CODEs and  CodeIDs

In [17]:
df_ravel.drop(['HP_SYNONYM'],axis=1,inplace=True)
df_ravel.rename(columns={'HP_ID':'CODE_HPO','MPO_URI':'CODE_MP'},inplace=True)
df_ravel.head(4)

,CODE_HPO,HP_LABEL,CODE_MP,MPO_LABEL
0,HP:0000011,neurogenic bladder,MP:0005302,neurogenic bladder
1,HP:0000023,inguinal hernia,MP:0006077,inguinal hernia
2,HP:0000028,cryptorchidism,MP:0002286,cryptorchism
3,HP:0000033,"ambiguous genitalia, male",MP:0002160,abnormal reproductive system morphology | abno...


In [18]:
df_ravel.shape

(1221, 4)

## There are already HPO terms in UMLS. We only need to create new CUIs/CODEs/CodeIDs for HPO terms that arent in there. But first we need to get HPO CUI/CODEs from UMLS and merge them into the dataframe (merge in CUIs on HPO Code).
match (n:Code)--(m:Concept) where n.SAB = 'HPO' return n.CODE as HPO_CODE,m.CUI AS HPO_CONCEPT   
^^^Downloaded from neo4j desktop app and saved as umls-hpo-code-concepts.csv

In [19]:
# Load UMLS HPO CODEs-CUIs
#umls_hpo = pd.read_csv('/Users/stearb/desktop/R03_local/data/umls-hpo-code-concepts.csv')
#umls_hpo.rename(columns={'HPO_CODE':'CODE_HPO','HPO_CONCEPT':'CUI_HPO'},inplace=True)
#umls_hpo.head()

In [20]:
df_ravel['CodeID_MP'] = ['MP '+i for i in df_ravel['CODE_MP']]
df_ravel['CodeID_HPO'] = ['HPO '+i for i in df_ravel['CODE_HPO']]

#CUI_LEN = 14
#df_ravel['CUI_MP']  = ['KC' + str(int(hashlib.sha256(uid.encode('utf8')).hexdigest(),base=16))[:CUI_LEN] for uid in df_ravel['CODE_MP']]


#CUIs_mp = CUIbase64(df_ravel['CodeID_MP'])
#df_ravel['CUI_MP'] = [i for i in CUIs_mp]

# dont need this. see venn diagram below. all HPO terms we are using are in UMLS already
# df_ravel['CUI_HPO']  = ['KC' + str(np.abs(hash(uid)))[:CUI_LEN] for uid in df_ravel['CODE_HPO']]

assert len(df_ravel['CODE_MP'].unique())  ==  len(df_ravel['CodeID_MP'].unique()) 
assert len(df_ravel['CODE_HPO'].unique())  ==  len(df_ravel['CodeID_HPO'].unique()) 

In [21]:
print(df_ravel['CODE_HPO'].unique().shape)
#umls_hpo['CODE_HPO'].unique().shape

(719,)


In [22]:
#from matplotlib_venn import venn2

#venn2([set(df_ravel['CODE_HPO']),
#       set(umls_hpo['CODE_HPO'])])

# all 719 HPO terms we have are already in UMLS so we dont need to create any new CUIs
# Just use the CODE_HPO (aka the hpo term) to map the UMLS CUIs to my df_ravel.

In [23]:
df_ravel.head(3)

,CODE_HPO,HP_LABEL,CODE_MP,MPO_LABEL,CodeID_MP,CodeID_HPO
0,HP:0000011,neurogenic bladder,MP:0005302,neurogenic bladder,MP MP:0005302,HPO HP:0000011
1,HP:0000023,inguinal hernia,MP:0006077,inguinal hernia,MP MP:0006077,HPO HP:0000023
2,HP:0000028,cryptorchidism,MP:0002286,cryptorchism,MP MP:0002286,HPO HP:0000028


In [25]:
# check umls hp and mp code id format
codes  = pd.read_csv('/Users/stearb/Desktop/DESKTOP_TRANSFER/DataDistilleryDistributions/DataDistillery10Sept2023/DataDistillery10September2023/CODEs.csv')

In [24]:
codes[codes['CODE'].str.startswith('HP')].head(2)

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
codes[codes['CodeID:ID'].str.startswith('MP')].head(2)

# Make nodes file

In [27]:
nodes_mp = df_ravel[['CodeID_MP','MPO_LABEL']]
nodes_hpo = df_ravel[['CodeID_HPO','HP_LABEL']]
nodes_mp.columns = nodes_hpo.columns = ['node_id','node_label']
nodes = pd.concat([nodes_mp,nodes_hpo],axis=0).drop_duplicates()

# dont need to include terms these ontologies are both already in umls 
#nodes['node_label'] = np.nan

# other colls can be nan as well
#nodes['node_synonyms'] = np.nan
#nodes['node_namespace'] = np.nan
#nodes['node_dbxrefs'] = np.nan
#nodes['node_definition'] = np.nan

In [34]:
nodes = fill_missing_cols(nodes)
nodes

,node_id,node_label,unit,value,node_namespace,lowerbound,node_dbxrefs,upperbound,node_synonyms,node_definition
0,MP:0005302,neurogenic bladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MP:0006077,inguinal hernia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MP:0002286,cryptorchism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MP:0002160,abnormal reproductive system morphology | abno...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MP:0001145,abnormal reproductive system morphology | abno...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1210,HP:0410277,sternal pit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,HP:0410278,pituitary gland cyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,HP:0410279,atrophic pituitary gland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,HP:0410287,intrathoracic hemangioma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
nodes['node_id'] = [i.replace('MP ','') if i.startswith('MP') else i for i in nodes['node_id'] ]
nodes

,node_id,node_label,unit,value,node_namespace,lowerbound,node_dbxrefs,upperbound,node_synonyms,node_definition
0,MP:0005302,neurogenic bladder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MP:0006077,inguinal hernia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MP:0002286,cryptorchism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MP:0002160,abnormal reproductive system morphology | abno...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MP:0001145,abnormal reproductive system morphology | abno...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1210,HP:0410277,sternal pit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,HP:0410278,pituitary gland cyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1214,HP:0410279,atrophic pituitary gland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1216,HP:0410287,intrathoracic hemangioma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
nodes['node_id'] = [i.replace('HPO ','') if i.startswith('HPO') else i for i in nodes['node_id'] ]

nodes

,node_id,node_label
0,MP:0005302,neurogenic bladder
1,MP:0006077,inguinal hernia
2,MP:0002286,cryptorchism
3,MP:0002160,abnormal reproductive system morphology | abno...
4,MP:0001145,abnormal reproductive system morphology | abno...
...,...,...
1210,HP:0410277,sternal pit
1213,HP:0410278,pituitary gland cyst
1214,HP:0410279,atrophic pituitary gland
1216,HP:0410287,intrathoracic hemangioma


## Make edges file

In [36]:
#df_ravel['predicate'] = 'RO:0002603' # 'is approximately equivalent to'     OLD rel: #'has_human_phenotype'

df_ravel['predicate'] = 'http://purl.obolibrary.org/obo/RO_0002603'

edges = df_ravel[['CodeID_MP','predicate','CodeID_HPO']]
edges.columns = ['subject','predicate','object']
edges.head(3)

,subject,predicate,object
0,MP MP:0005302,http://purl.obolibrary.org/obo/RO_0002603,HPO HP:0000011
1,MP MP:0006077,http://purl.obolibrary.org/obo/RO_0002603,HPO HP:0000023
2,MP MP:0002286,http://purl.obolibrary.org/obo/RO_0002603,HPO HP:0000028


In [37]:
edges['subject'] = edges['subject'].str.replace('MP ','')
edges['object'] = edges['object'].str.replace('HPO ','')
edges

,subject,predicate,object
0,MP:0005302,http://purl.obolibrary.org/obo/RO_0002603,HP:0000011
1,MP:0006077,http://purl.obolibrary.org/obo/RO_0002603,HP:0000023
2,MP:0002286,http://purl.obolibrary.org/obo/RO_0002603,HP:0000028
3,MP:0002160,http://purl.obolibrary.org/obo/RO_0002603,HP:0000033
4,MP:0001145,http://purl.obolibrary.org/obo/RO_0002603,HP:0000033
...,...,...,...
1216,MP:0002006,http://purl.obolibrary.org/obo/RO_0002603,HP:0410287
1217,MP:0014086,http://purl.obolibrary.org/obo/RO_0002603,HP:0410287
1218,MP:0002947,http://purl.obolibrary.org/obo/RO_0002603,HP:0410287
1219,MP:0002092,http://purl.obolibrary.org/obo/RO_0002603,HP:0500049


# Save files 

In [38]:
#edges.to_csv('/Users/stearb/Desktop/DataDistilleryFiles/phenotype_mappings/edges_phenoMapping.tsv',sep='\t',index=False)

# save edges with this name for alan/js's script
edges.to_csv('/Users/stearb/Desktop/DESKTOP_TRANSFER/DataDistilleryFiles/phenotype_mappings/OWLNETS_edgelist.txt',
             sep='\t',index=False)

nodes.to_csv('/Users/stearb/Desktop/DESKTOP_TRANSFER/DataDistilleryFiles/phenotype_mappings/OWLNETS_node_metadata.txt',
             sep='\t',index=False)

In [30]:
len(edges)

1221

# OLD WAY

### Merge in UMLS  HPO CUIs

In [23]:
#df_merge = pd.merge(left=df_ravel,right=umls_hpo.drop_duplicates(['CODE_HPO']),on='CODE_HPO',how='inner')
#df_merge.head()

Save CUIs

In [32]:
# Dont need to  save  df_merge['CUI_HPO'], they are already in UMLS
#pd.DataFrame(df_merge['CUI_MP'].drop_duplicates(),columns=['CUI']).to_csv('CUIs_phenomapping.csv',index=False) 

#! mv CUIs_phenomapping.csv /Users/stearb/desktop/R03_local/data/ingest_files/hpo_mp_mapping/

In [33]:
## Code for generating all HPO terms containing the word heart
#heart_mp_terms = pd.read_csv('/Users/stearb/Desktop/R03_local/data/gene_sets/heart_mp_terms.csv',header=None)
#q=df_merge[['CODE_MP','CODE_HPO']]

#q[q['CODE_MP'].isin(q['CODE_MP'].isin(heart_mp_terms))]

#t  = (list(heart_mp_terms.values))
#heart_terms = [item for sublist in t for item in sublist]

#df_merge['CODE_MP'].isin(heart_mp_terms)
#set(df_merge['CODE_MP']).intersection(set(heart_mp_terms))


Save CUI-CUIs ( Same relationship SAB for both HPO->MP & MP->HPO )

In [34]:
CUI_CUI = df_merge[['CUI_HPO','CUI_MP']]

# Add SAB and relationship type (:TYPE) so the df matches the format of the UMLS import files.
CUI_CUI[':TYPE'] = 'has_mouse_phenotype'
CUI_CUI['SAB'] = 'HPO__MP'

# Reverse the columns to create the inverse relationship, has_human_phenotype
CUI_CUI_inverse = df_merge[['CUI_MP','CUI_HPO']]
CUI_CUI_inverse[':TYPE'] = 'has_human_phenotype'
CUI_CUI_inverse['SAB'] = 'HPO__MP'

# Now combine both (need to change column names first so they can be concatenated) and save.
CUI_CUI.rename(columns={'CUI_HPO':':START_ID','CUI_MP':':END_ID'},inplace=True)
CUI_CUI_inverse.rename(columns={'CUI_HPO':':END_ID','CUI_MP':':START_ID'},inplace=True)

pd.concat([CUI_CUI,CUI_CUI_inverse]).to_csv(
            '/Users/stearb/desktop/R03_local/data/ingest_files/hpo_mp_mapping/CUI-CUI_phenomapping.csv',index=False) 
 

Save CUI-CODEs

In [35]:
#No need to save CUI-CODE relationships for the HPO terms, they are  already in UMLS
mp_cui_codes = df_merge[['CUI_MP','CodeID_MP']]

mp_cui_codes_reformat = mp_cui_codes.rename(columns={'CUI_MP':'CUI','CodeID_MP':'CODE'}).drop_duplicates()

mp_cui_codes_reformat.to_csv('/Users/stearb/desktop/R03_local/data/ingest_files/hpo_mp_mapping/CUI-CODEs_phenomapping.csv',index=False)

Save CODEs

In [37]:
# hpo_codes = df_merge[['CodeID_HPO','CODE_HPO']]   # these are already in UMLS, no need to add them.
mp_codes = df_merge[['CodeID_MP','CODE_MP']]
mp_codes['SAB'] = 'MP'
mp_codes.rename(columns={'CodeID_MP':'CodeID','CODE_MP':'CODE'},inplace=True)
mp_codes  = mp_codes[['CodeID','SAB','CODE']]
mp_codes.to_csv('/Users/stearb/desktop/R03_local/data/ingest_files/hpo_mp_mapping/CODEs_phenomapping.csv',index=False)


# End of data formatting for the HPO-MP mappings

### Save HPO Terms to generate gene sets for
We can only generate gene sets for each HPO terms that we have mapped to MP terms, so whatever HPO terms we end up with, we will have to drop the ones that are not currently mapped to an MP term.

In [ ]:

# Old way where we just took any term that had 'heart' in it.
terms2query = df_ravel[df_ravel['MPO_LABEL'].str.contains('heart',case=False)]
terms2query[['HP_ID','HP_LABEL']].to_csv('/Users/stearb/Desktop/R03_local/data/gene_sets/hpo_terms/heart_hpo_terms.csv',index=None)

# New way is in /results_queries/get_gene_sets.ipynb

In [1]:
#for m,i in enumerate(mp_codes['CODE'].values):
#    if i  == 'MP:0003923':
#        print('--------');print(m,i);print('--------')

In [30]:
df_merge[df_merge['CODE_HPO'] == 'HP:0025579']

,CODE_HPO,CODE_MP,CodeID_MP,CodeID_HPO,CUI_MP,CUI_HPO
993,HP:0025579,MP:0003923,MP MP:0003923,HPO HP:0025579,TVAgTVA6MDAwMzkyMw==,C4703395


### Save whole df so we can check that there are no collisions with these CUIs and CUIs from the other steps

In [31]:
df_merge.to_csv('/Users/stearb/desktop/R03_local/data/UI_check/hpo_mp_mappings.csv',index=False)

#### Whats the overlap of these MP terms with the MP terms from step 2 (geno-pheno)?
#### Dont need to worry about this because we created a MP nodes file 'MP_TERMS_NODES.csv' that contains mp terms from both lists

In [31]:
g2p = pd.read_csv('/Users/stearb/desktop/R03_local/data/ingest_files/genopheno/geno2pheno_mapping.csv',index_col=0)

In [18]:
g2p['mp_term_id'].unique().shape[0]

740

In [19]:
# Whats the overlap?
mp_overlap = set(df_ravel['MPO_URI'].unique()).intersection(set(g2p['mp_term_id'].unique()))
len(mp_overlap)

51

In [87]:
# Really only 51 MP terms in both lists?
a=0
for k in g2p['mp_term_id'].unique():
    if k in df_ravel['MPO_URI'].unique():
        a+=1
print(a)

51
